In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_11111111111111111111111111111111 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='********************************************',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_73fb2e24d42a4e77af59e451e92f34c8.get_object(Bucket='smartbridge-donotdelete-pr-oz3awn6bck6a29',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )


In [ ]:
dataset = pd.read_csv(body)
dataset.head()

# Data Preprocessing

In [ ]:
# Replacing the nan values with mean values
#dataset = dataset.fillna(value = dataset.mean())
dataset = dataset.dropna(how = 'any')
# Dropping the year column
dataset = dataset.drop('Year', axis = 1)

In [ ]:
# Encoding the Status column
le = LabelEncoder()
le.fit(dataset['Status'])
dataset['Status'] = le.transform(dataset['Status'])
dataset.head()

In [ ]:
# Encoding the Country column
le2 = LabelEncoder()
le2.fit(dataset['Country'])
dataset['Country'] = le2.transform(dataset['Country'])

In [ ]:
dataset.head(1)

# Heat map of correlation between features and Life Expectancy

In [ ]:
# Plotting a heat map representing correlation of each column with Life Expectancy
# Setting the figure size
plt.figure(figsize = (15,10))
sns.heatmap(dataset.corr(), annot = True)

# Dependent and Independent variables

In [ ]:
# Creating Independent and Dependent variables
y = dataset['Life expectancy '].values

X = dataset.drop(columns = ['Life expectancy ']).values

# Splitting the dataset into training and testing data

In [ ]:
# Splitting the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Linear Regression

In [ ]:
# Applying simple Linear Regression
regr1 = LinearRegression()
regr1.fit(X_train, y_train)
regr1.coef_

In [ ]:
y_pred1 = regr1.predict(X_test)
print('Simple Linear Regression : ')
print('Mean Absolute Error : ',mae(y_test, y_pred1))
print('R_square score: %.4f' % r2_score(y_test, y_pred1))

# Random Forest Regression

In [ ]:
# Implementing Random Forest Regression
regr2 = RandomForestRegressor(n_estimators = 800, random_state = 0)

In [ ]:
regr2.fit(X_train, y_train)

In [ ]:
X_test[0]

In [ ]:
y_pred2 = regr2.predict(X_test)

print('Random Forest Regression : ')
print('Mean Absolute Error : ',mae(y_test, y_pred2))
print('R_square score : %4f' % r2_score(y_test, y_pred2))

In [ ]:
regr2.predict(X_test)

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials = {
  "apikey": "********************************************",
  "iam_apikey_description": "Auto-generated for key e4d27bff-5a89-4fb0-8ba6-d3da30ff13ae",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/4a6ecb30c7da452c8aa6cbf4dabbc983::serviceid:ServiceId-98af2fca-5093-45e6-8c5c-f264eacceedd",
  "instance_id": "************************************",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Akshat",
    client.repository.ModelMetaNames.AUTHOR_EMAIL: "",
    client.repository.ModelMetaNames.NAME : "dont_expect_life"
}

In [ ]:
model_artifact = client.repository.store_model(regr2, meta_props = model_props)

In [ ]:
guid = client.repository.get_model_uid(model_artifact)
guid

In [ ]:
deploy = client.deployments.create(guid, name = "no_life")

In [ ]:
client.deployments.list()